<a href="https://colab.research.google.com/github/dahlia52/ESAA_YB/blob/main/Project/YB_team1_mini_project2_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DACON 영화 관객수 예측 경진대회**

https://dacon.io/competitions/open/235536/data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/Data/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Data/movies_test.csv')
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [ ]:
# 결측치가 많은 데이터 제거
train = train.drop(['dir_prev_bfnum'],axis = 1)
test =  test.drop(['dir_prev_bfnum'],axis = 1)

In [ ]:
# 감독명 : 너무 다양해서 제거
train = train.drop(['director'],axis = 1)
test = test.drop(['director'],axis = 1)

In [ ]:
# 제목 : 의미가 없기 때문에 제거
train = train.drop(['title'],axis= 1)
test = test.drop(['title'],axis= 1)

In [ ]:
train.distributor.value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

In [ ]:
# 상위 5개의 배급사를 제외하고 '기타'로처리
distributor_list = train.distributor.value_counts()[:5]
def func(distributor):
    if distributor in distributor_list:
        return distributor
    else:
        return '기타'

train['distributor'] = train['distributor'].apply(lambda x : func(x))
test['distributor'] = test['distributor'].apply(lambda x : func(x))

In [ ]:
# 개봉일을 바탕으로 년,월 변수 생성
train['년'] = train['release_time'].apply(lambda x: int(x[:4]))
train['월'] = train['release_time'].apply(lambda x: int(x[5:7]))
train =  train.drop(['release_time'],axis = 1)

test['년'] = test['release_time'].apply(lambda x: int(x[:4]))
test['월'] = test['release_time'].apply(lambda x: int(x[5:7]))
test =  test.drop(['release_time'],axis = 1)

In [ ]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
#정규화
y_train = train['box_off_num']
X_train = train.drop('box_off_num', axis=1)
X_test = test

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# MinMaxScaler
minmax_scaler = preprocessing.MinMaxScaler()
norm_fit = minmax_scaler.fit(X_train)
X_train_norm = norm_fit.transform(X_train)
X_test_norm = norm_fit.transform(X_test)

- 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

params = {'n_estimators' : [100,500], 'learning_rate' : [0.05,0.1]}

gbm = GradientBoostingRegressor(random_state=0)
grid_cv = GridSearchCV(gbm, param_grid=params, cv=2, verbose=1, scoring='neg_mean_squared_error')
grid_cv.fit(X_train, y_train)

print('최적의 하이퍼 파라미터 : \n', grid_cv.best_params_)
print('최고 예측 정확도 : ', grid_cv.best_score_)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
최적의 하이퍼 파라미터 : 
 {'learning_rate': 0.05, 'n_estimators': 100}
최고 예측 정확도 :  -2583654874923.142


In [ ]:
# MinMaxScaler
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05)
gbm.fit(X_train_norm, y_train)
gbm_pred = gbm.predict(X_test_norm)

In [ ]:
# 음수 예측값들을 0으로 변경
gbm_pred[gbm_pred<0]=0

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Data/submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [ ]:
submission['box_off_num'] = gbm_pred

In [ ]:
submission

,title,box_off_num
0,용서는 없다,2.300392e+06
1,아빠가 여자를 좋아해,1.648844e+06
2,하모니,1.707688e+06
3,의형제,2.407292e+06
4,평행 이론,1.039696e+06
...,...,...
238,해에게서 소년에게,9.457231e+03
239,울보 권투부,1.544556e+04
240,어떤살인,5.035925e+05
241,말하지 못한 비밀,0.000000e+00


In [ ]:
submission.to_csv('GBM_MinMax.csv',index = False)

# GBM RMSE

MinMax: 1265380.591066857	